In [0]:
WITH sales_with_details AS (
  SELECT
    s.sale_id,
    s.product_id,
    s.customer_id,
    s.quantity,
    c.region,
    p.product_name
  FROM sales s
  JOIN customers c
    ON s.customer_id = c.customer_id
  JOIN products p
    ON s.product_id = p.product_id
),
aggregated_sales AS (
  SELECT
    region,
    product_name,
    SUM(quantity) AS total_quantity
  FROM sales_with_details
  GROUP BY region, product_name
),
ranked_products AS (
  SELECT
    region,
    product_name,
    total_quantity,
    dense_rank() OVER (
      PARTITION BY region
      ORDER BY total_quantity DESC
    ) AS rank_in_region
  FROM aggregated_sales
)
SELECT
  region,
  product_name,
  total_quantity,
  rank_in_region
FROM ranked_products
WHERE total_quantity > 0 and rank_in_region < 3;
